# Прогноз результатов матчей на основе TrueSkill

Загружаем нужные библиотеки.

In [13]:
import pandas as pd
import trueskill
import json

from math import sqrt, log
from trueskill import BETA
from trueskill.backends import cdf

Загружаем информацию про результаты игр, скачанную с http://dotascience.com/ и сразу сортируем по Match_ID.

In [2]:
matches_info = pd.read_csv('selected_team_matches.csv').sort(['match_id'])


/home/anton/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [3]:
f = open('selected_team_match_info.jsonlines', 'r')
alldata = f.readlines()
f.close()

In [4]:
teams_data = pd.DataFrame(columns = ['time', 'dire', 'radiant', 'winner'])

for match_string in alldata:
    match = json.loads(match_string)
    res = [0,'','','']
    try:
        res[0] = match[u'start_time']
        res[1] = match[u'dire_name']
        res[2] = match[u'radiant_name']
        if match['radiant_win']:
            res[3] = 'RADIANT'
        else:
            res[3] = 'DIRE'
        #print(res)
    except:
        pass
    teams_data.loc[len(teams_data)] = res

In [5]:
#teams_data.sort(['time'])
teams_data.dire.value_counts()

Evil Geniuses                       445
CDEC Gaming                         133
Team Liquid                         133
compLexity Gaming                   129
LGD-GAMING                          128
Virtus.pro                          121
Vici Gaming                         119
Alliance                            113
MVP Phoenix                         108
EHOME                               105
Team Secret                         103
Newbee                               96
Fnatic                               84
Team Empire                          67
Invictus Gaming                      58
OG Dota2                             57
                                     55
Natus Vincere                        53
Cloud9 G2A                           47
TeamMalaysia                         42
Team. Spirit                         39
Team Archon                          39
-Ninjas-in-Pyjamas-                  38
Vega Squadron                        35
(monkey) Business                    31


Инициализируем параметры Trueskill используя параметры по умолчанию, кроме вероятность ничейного результата, у нас не бывает ничьих.

In [ ]:
trueskill.setup(draw_probability=0)

Создадим словарик, где будем хранить рейтинги. И обучим нашу систему. Для каждого матча мы определим победителя и пересчитаем рейтинги комманд. Если команда раньше не встречалась задаем ей рейтинг по умочанию. В нашей простейшей модели мы считаем, что каждая команда это один виртуальный игрок.

In [ ]:
Rates = {}
for index, row in matches_info.iterrows():
    team1 = row['radiant']
    team2 = row['dire']
    if team1 not in Rates:
        Rates[team1] = trueskill.Rating()
    if team2 not in Rates:
        Rates[team2] = trueskill.Rating()
    if row['winner'] == 'RADIANT':
        Rates[team1], Rates[team2] = trueskill.rate_1vs1(Rates[team1], Rates[team2])
    else:
        Rates[team2], Rates[team1] = trueskill.rate_1vs1(Rates[team2], Rates[team1])
        

In [6]:
Rates_rec_data = {}

for index, row in teams_data.iterrows():
    team1 = row['radiant']
    team2 = row['dire']
    if team1 not in Rates_rec_data:
        Rates_rec_data[team1] = trueskill.Rating()
    if team2 not in Rates_rec_data:
        Rates_rec_data[team2] = trueskill.Rating()
    if row['winner'] == 'RADIANT':
        Rates_rec_data[team1], Rates_rec_data[team2] = trueskill.rate_1vs1(Rates_rec_data[team1], Rates_rec_data[team2])
    else:
        Rates_rec_data[team2], Rates_rec_data[team1] = trueskill.rate_1vs1(Rates_rec_data[team2], Rates_rec_data[team1])
        

Теперь мы можем, например, насладится рейтингом команты Virtus.pro, которая скрывается под тагом 'VP 2' 

In [21]:
Rates_rec_data['CDEC Gaming' ]

trueskill.Rating(mu=28.540, sigma=0.825)

Попробуем оценить вероятность победы Virtus.pro, например, в матче с Evil Genius.
К сожалению в библиотеке trueskill, нет возможности оценить вероятность, но мы же знаем из описания,
что рейтинги тут нормально распределенные. Так что сделаем функцию, которая оценит вероятность, 
что сила игры первой команды, будет больше чем у второй.

In [10]:
def win_probability(player_rating, opponent_rating):
    delta_mu = player_rating.mu - opponent_rating.mu
    denom = sqrt(2 * (BETA * BETA) + pow(player_rating.sigma, 2) + pow(opponent_rating.sigma, 2))
    return cdf(delta_mu / denom)

Ура, теперь мы можем оценить шансы.

In [ ]:
win_probability(Rates['VP 2'], Rates['EG'])

Система сказала, что шансы почти равные. Теперь полученную оценку можно загрузить на сайт в качетве прогноза. 
Как это сделать расказанно [тут](https://github.com/romovpa/dotascience-hackathon/blob/master/README.md).

In [11]:
def sort_by_rating(team):
    return float(team[1])

def sort_by_rating(R):
    rates_simple = []
    print(type(R))
    for k in R:
        #print(k)
        row = [k, R[k].mu, R[k].sigma]
        rates_simple.append(row)

    #pd.DataFrame(rates_simple).to_csv('rates_simple.csv')

    #sorted_rates_simple = sorted(rates_simple, key = sort_by_rating)
    return rates_simple

In [12]:
Rates_rec_data['MVP Phoenix']

trueskill.Rating(mu=25.455, sigma=0.835)

In [ ]:
#sort_by_rating(Rates_rec_data)

In [13]:
Rates_rec_data['Recovery Position'].sigma

5.919599884281653

In [14]:
current_major_data = pd.read_json('major_results.json').transpose()

In [15]:
Rates_major = {}

for index, row in current_major_data.iterrows():
    team1 = row['radiant']
    team2 = row['dire']
    if team1 not in Rates_major:
        Rates_major[team1] = trueskill.Rating()
    if team2 not in Rates_major:
        Rates_major[team2] = trueskill.Rating()
    if row['winner'] == 'RADIANT':
        Rates_major[team1], Rates_major[team2] = trueskill.rate_1vs1(Rates_major[team1], Rates_major[team2])
    else:
        Rates_major[team2], Rates_major[team1] = trueskill.rate_1vs1(Rates_major[team2], Rates_major[team1])
        

In [25]:
sort_by_rating(Rates_rec_data)

<type 'dict'>


NameError: global name 'sorted_rates_simple' is not defined

In [12]:
pd.DataFrame(sort_by_rating(Rates_rec_data)).to_csv('rank_.csv')

<type 'dict'>


In [9]:
pd.DataFrame(Rates_rec_data)

TypeError: object of type 'Rating' has no len()

In [14]:
trueskill.Rating()

trueskill.Rating(mu=25.000, sigma=8.333)